In [26]:
import re
import unicodedata

import pandas as pd
import spacy
from rapidfuzz import fuzz
from sklearn.model_selection import train_test_split

# === 1. Inițializare ===
nlp = spacy.load("ro_core_news_sm")


# === 2. Funcții utilitare ===
def remove_diacritics(text):
    return ''.join(
        c for c in unicodedata.normalize('NFD', text)
        if unicodedata.category(c) != 'Mn'
    )


def clean_text(text):
    if pd.isna(text):
        return ""
    text = text.replace("ş", "ș").replace("ţ", "ț").lower()
    text = re.sub(r"[^a-zăîâșț0-9\s]", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    return remove_diacritics(text)


def normalize_phrase(phrase):
    phrase = clean_text(phrase)
    doc = nlp(phrase)
    lemmatized = ' '.join(
        token.lemma_ for token in doc
        if not token.is_punct
        and not token.is_space
        and token.pos_ not in ["ADP", "DET", "CCONJ", "SCONJ", "PART", "PRON"]
    )
    return remove_diacritics(lemmatized).lower()


def compute_scor_medical(row):
    scor = 0

    # === 1. Factori demografici și antropometrici ===
    if row.get("Vârstă", 0) > 45:
        scor += 1
    if row.get("IMC", 0) > 30:
        scor += 2
    if row.get("obezitate abdominala", 0) == 1:
        scor += 2

    # === 2. Diagnostic sau condiții medicale ===
    scor += int(row.get("rezistenta la insulina", 0)) * 4
    scor += int(row.get("prediabet", 0)) * 6
    scor += int(row.get("diabet zaharat tip 2", 0)) * 10
    scor += int(row.get("ficat gras", 0)) * 2
    scor += int(row.get("hipertensiune arteriala", 0))
    scor += int(row.get("dislipidemie (grăsimi crescute in sânge)", 0))

    # === 3. Simptome subiective și comportamente metabolice ===
    scor += int(row.get("slăbesc greu", 0))
    scor += int(row.get("mă îngraș ușor", 0))
    scor += int(row.get("depun grasime in zona abdominala", 0))
    scor += int(row.get("oboseala permanenta", 0)) * 2
    scor += int(row.get("urinare nocturna", 0)) * 2
    scor += int(row.get("lipsa de energie", 0)) * 2
    scor += int(row.get("pofte de dulce", 0)) * 2
    scor += int(row.get("foame greu de controlat", 0)) * 2

    # === 4. Sex-specific ===
    sex = str(row.get("Ești", "")).strip().lower()
    talie = row.get("Care este circumferința taliei tale, măsurata deasupra de ombilicului?", 0)

    try:
        talie_val = float(talie)
    except:
        talie_val = 0  # în caz că e NaN sau text

    if sex == "femeie":
        scor += int(row.get("sindromul ovarelor polichistice", 0)) * 2
        if isinstance(row.get("labels"), list) and "ginecologic_hormonal" in row["labels"]:
            scor += 2
        if talie_val > 80:
            scor += 2
        if talie_val > 120:
            scor += 3

    elif sex == "barbat":
        if talie_val > 94:
            scor += 2
        if talie_val > 120:
            scor += 3  # prag pentru bărbați

    # === 5. Etichete NLP (semnale indirecte de risc) ===
    if isinstance(row.get("labels"), list):
        scor += 4 if "metabolic_endocrin" in row["labels"] else 0
        scor += 2 if "cardio_vascular" in row["labels"] else 0
        scor += 1 if "gastro_hepato_renal" in row["labels"] else 0
        scor += 1 if "inflamator_autoimun" in row["labels"] else 0
        scor += 1 if "neuro_psiho_energie" in row["labels"] else 0

    return scor


# === 3. Încărcare și preprocesare text ===
df = pd.read_csv("processed/processed_data.csv", delimiter=';')
df.columns = df.columns.str.strip().str.replace('*', '', regex=False)

# Coloane de interes
col1 = "Ce alte simptome sau boli prezinți?"
col2 = "In prezent, care este cea mai mare provocare a ta? Ce crezi ca te împiedica sa slăbești si sa ai o stare buna de sănătate? "

# Curățare text
df[col1] = df[col1].fillna("").apply(clean_text)
df[col2] = df[col2].fillna("").apply(clean_text)

# Combinare + lematizare
df["text_lemmatized"] = (df[col1] + " " + df[col2]).apply(normalize_phrase)

# === 4. Dicționar categorii medicale ===
general_symptom_categories = {
    "metabolic_endocrin": [
        "diabet", "diabet zaharat", "prediabet", "rezistenta insulina", "insulina mare", "pofta dulce", "pofte dulce",
        "glicemie mare",
        "glicemie crescuta", "hipoglicemie", "zahar scazut",
        "colesterol mare", "colesterol crescut", "dislipidemie", "trigliceride mari", "grasimi sange",
        "obezitate", "supraponderal", "greutate mare",
        "hipotiroidism", "tiroida lenesa", "hipotiroidie", "hipertiroidism", "tiroida activa", "tiroida marita",
        "tiroida autoimuna",
        "tiroidita hashimoto", "tiroida hashimoto", "boala autoimuna tiroidiana", "probleme tiroida",
        "noduli tiroidieni", "noduli tiroida", "nodul tiroida", "tiroida nodulara",
        "sindrom cushing", "cortizol crescut",
        "par exces", "hirsutism", "gura uscata", "sete excesiva", "mi-e sete tot timpul",
        "foame permanenta", "slabesc fara motiv", "slabire inexplicabila", "vedere incetosata", "vad ceata",
        "probleme vedere"
        "urinari frecvente", "fac pipi des", "urinari nocturne", "ma trezesc noaptea merg baie",
        "infectii urinare frecvente", "rani vindeca greu",
        "intepaturi maini picioare", "furnicaturi", "amorteala picioare",
        "ma mananca pielea fara motiv", "piele uscata", "acnee hormonala", "infectii fungice frecvente",
        "candidoza recurenta"
    ],
    "cardio_vascular": [
        "hipertensiune", "tensiune mare", "tensiune crescuta", "tensiune arteriala ridicata",
        "tahicardie", "batai rapide inimii", "puls mare", "aritmii", "batai neregulate",
        "simt inima bate tare", "angina", "durere piept", "presiune toracica", "infarct",
        "infarct miocardic", "atac cord", "trombofilia", "cheaguri sange", "coagulare excesiva",
        "edeme", "picioare umflate", "retentie apa", "apa picioare", "umflaturi glezne",
        "am tensiune mare", "mi ridica tensiunea", "imi bate inima tare", "am inima agitata",
        "imi simt inima gat", "imi tremura pieptul", "am stari lesin", "am ameteli dese",
        "mi intuneca vederea cand ma ridic", "simt apasare piept", "ma strange piept",
        "am dureri piept", "ma doare inima", "am senzatie sufocare", "nu pot respira bine",
        "ma sufoc", "obosesc repede", "mi umfla picioarele", "am glezne umflate",
        "tensiune oscilanta", "am cap greu", "am dureri ceafa",
        "mi inroseste fata", "ma infierbant repede", "sforai puternic", "apnee somn", "apnee",
        "transpir excesiv", "ma ia calduri senin", "am colesterol mare", "am trigliceride mari",
        "mananc sarat", "fumez zilnic", "fumez mult"
    ],
    "neuro_psiho_energie": [
        "epuizare", "lipsa energie", "astenie", "slabiciune", "migrena", "migrene", "durere cap", "cap greu",
        "cefalee", "insomnie", "nu pot dormi", "somn agitat",
        "anxietate", "neliniste", "atacuri panica", "nervozitate",
        "stres crescut", "iritabilitate", "obosit cronic", "oboseala"
    ],
    "gastro_hepato_renal": [
        "balonare", "burta umflata", "gaze", "constipatie", "nu merg toaleta", "scap rar", "diaree",
        "scaune moi", "merg des", "reflux", "arsuri stomac", "acid stomac", "regurgitare",
        "sindromul intestinului iritabil", "colon iritabil", "probleme digestive",
        "ficat gras", "steatoza hepatica", "hepatita cronica", "probleme ficat",
        "ficat marit", "hepatomegalie", "boala ficat", "probleme hepatice", "boala cronica hepatica", "ciroza",
        "boala cronica rinichi", "rinichi slabi", "probleme renale", "insuficienta renala"
    ],
    "ginecologic_hormonal": [
        "amenoree", "lipsa menstruatiei", "menstruatie oprita", "menopauza", "dereglare ciclu menstrual",
        "sunt menopauza", "premenopauza", "simptome menopauza",
        "ovare polichistice", "sindrom ovare polichistice", "sop",
        "endometrioza", "durere menstruala", "dureri pelvine", "adenomioza",
        "sangerari abundente", "probleme hormonale", "nivel estrogen",
        "estrogen scazut", "estrogen crescut", "sindrom premenstrual",
        "pms", "balonare ciclu", "crampe menstruale"
    ],
    "inflamator_autoimun": [
        "inflamatie cronica", "markeri inflamatori", "vsh crescut", "proteina c reactiva crescuta",
        "boala autoimuna", "lupus", "poliartrita reumatoida", "psoriazis", "boala celiaca", "boala crohn",
        "rectocolita ulcero-hemoragica", "spondilita anchilozanta", "boala behcet",
        "vasculita", "dermatita atopica", "eczeme cronice", "boala hashimoto", "boala inflamatorie",
        "durere articulara inflamatorie", "rigiditate articulara matinala", "umflaturi articulare", "rosata pielii",
        "febra inexplicabila", "inflamatie corp", "inflamatie"
    ]
}

# === 5. Normalizează toate frazele din categorii ===
normalized_categories = {
    cat: [normalize_phrase(phrase) for phrase in phrases]
    for cat, phrases in general_symptom_categories.items()
}


# === 6. Funcție de etichetare fuzzy ===
def label_row(text_lemma, threshold=90):  # daca cuvantul indeplineste threshold atunci se aplica eticheta
    labels = set()
    for category, phrases in normalized_categories.items():
        for phrase in phrases:
            if fuzz.partial_ratio(phrase, text_lemma) >= threshold:
                labels.add(category)
                break
    return list(labels)


# === 7. Aplică etichetarea ===
df["labels"] = df["text_lemmatized"].apply(label_row)
df["scor_medical"] = df.apply(compute_scor_medical, axis=1)

# === 7.5. Suprascriere etichete pe baza scorului medical ===
threshold_rezistenta = 20
threshold_prediabet = 30
threshold_diabet = 40


def infer_diagnosis_from_scor(row):
    if row.get("rezistenta la insulina", 0) == 0 and \
            row.get("prediabet", 0) == 0 and \
            row.get("diabet zaharat tip 2", 0) == 0:

        scor = row["scor_medical"]
        if scor >= threshold_diabet:
            row["diabet zaharat tip 2"] = 1
        elif scor >= threshold_prediabet:
            row["prediabet"] = 1
        elif scor >= threshold_rezistenta:
            row["rezistenta la insulina"] = 1
    return row


df = df.apply(infer_diagnosis_from_scor, axis=1)

# === 8. Salvează dataseturile ===
train, tmp = train_test_split(df, test_size=0.4, random_state=42)
val, test = train_test_split(tmp, test_size=0.5, random_state=42)
train.to_csv('datasets/train/train.csv', sep=';', index=False)
val.to_csv('datasets/validation/val.csv', sep=';', index=False)
test.to_csv('datasets/test/test.csv', sep=';', index=False)

print("Etichete extrase:", sorted(set(label for labels in df["labels"] for label in labels)))

Etichete extrase: ['cardio_vascular', 'gastro_hepato_renal', 'ginecologic_hormonal', 'inflamator_autoimun', 'metabolic_endocrin', 'neuro_psiho_energie']


In [27]:
train.head(3000)

,Vârstă,Ești,Unde locuiești?,Ocupatia,Care este înălțimea ta?,Care este greutatea ta actuala?,"Care este circumferința taliei tale, măsurata deasupra de ombilicului?",obezitate abdominala,rezistenta la insulina,prediabet,...,Care sunt obiectivele tale legate de sănătate? Ce îți dorești sa obții?,Greutatea la care vrei sa ajungi?,"In prezent, care este cea mai mare provocare a ta? Ce crezi ca te împiedica sa slăbești si sa ai o stare buna de sănătate?",De ce îți dorești sa slăbești?,Ai mai încercat in trecut sa slăbești?,Ce te-a împiedicat in trecut sa slăbești? De ce ai eșuat la alte încercări?,IMC,text_lemmatized,labels,scor_medical
3262,57,femeie,Constanta,Angajat,164.0,84.0,105.0,1,0,0,...,Sa slabesc 24 kg,60.0,tiroida lipsa de vointa ptr ca nu am obtinut r...,Sa nu ma mai doara spatele! Sa am energie! Sa ...,Da,Lipsa rezultatelor! Diete f stricte!,31.2,tiroidita autoimuna hipotiroidism alergie gene...,[metabolic_endocrin],19
2636,38,femeie,"Hereclean, Salaj",Angajat,165.0,87.0,95.0,0,0,0,...,"Scădere în greutate, modelare corporala, obice...",65 kg,privocare scaderea in greutate lipsa unui plan...,"Pt sanatate, aspect și stare de bine",1,"Stres, timp limitat, abandon alimentar, lipsa ...",32.0,prezint boala privocr scadere greutate lipsa p...,[inflamator_autoimun],8
2815,34,femeie,Satu Mare,Angajat,160.0,70.0,88.0,0,0,0,...,Sanatate,60,pofta de dulce,Sa nu fac diabet,0,Nu,27.3,pofta dulce,[metabolic_endocrin],17
1048,33,femeie,Bucuresti,Angajat,169.0,112.0,98.0,1,1,0,...,"Un mod de viata sanatos, si reducerea rezisten...",Sub 80kg,poftele,Sanatateeeeeee,1,Pt ca masuram kaloriile,39.2,boala pofta,[inflamator_autoimun],17
2994,49,femeie,"Vladimirescu, jud. Arad",Angajat,173.0,111.0,125.0,1,1,1,...,"80 kg în următoarele luni, sa mănânc sănătos, ...",80,emotiile lipsa de perseverenta,"Pentru a mă simți bine în corpul meu, pentru s...",1,Nu am rezistat mai mult de 2 luni cu dietă.,37.1,nod tiroida fara tratament puls crescut emotia...,[metabolic_endocrin],32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3444,60,femeie,galati,Angajat,160.0,72.0,97.0,1,1,0,...,eliminare rezistenta la insulina,60.0,sa revin la metabolism normal nu stiu de ce nu...,sa pot avea o viata sanatoasa,Da,"revenirea la vechile obiceiuri alimentare, pro...",28.1,avea reveni metabolism normal sti putea slabi,[],11
466,39,femeie,Pitesti,Angajat,165.0,61.4,82.0,1,1,0,...,"Să învăț să mănânc sănătos, să conștientizez c...","Credeam ca vreau sa ajung la 55-58 de kg, dar ...",anul trecut am fost diagnosticata cu tiroidita...,Deja am dat jos kg( cu ajutorul unui plan alim...,1,Anterior specialistului cu care colaborez în p...,22.6,tiroidita autoimuna an trecut avea fi diagnost...,[],16
3092,44,femeie,"comuna Sagna, judetul Neamt",Angajat,165.0,81.0,100.0,1,1,0,...,"mai putine medicamente de luat, scaderea tensi...",65 kg,pofta de dulciuri rontaielile in afara mesei p...,Pentru a ma simti bine cu mine,1,"delasarea, obiceiurile nocive, descurajarea ce...",29.8,colesterol mari alergie polen praf pofta dulci...,[metabolic_endocrin],20
3772,42,femeie,Usa,Angajat,164.0,71.0,98.0,1,1,0,...,Sa mi recapat energia si kg minus,60.0,sa scad nr anticorpilor si tsh ul pe care i am...,"Pt a imi capata energia, increderea in mine si...",Da,"Dupa cea de a doua nastere , la varsta de 35 a...",26.4,hashimoto hipotiroidism scadea nr anticorp tsh...,[metabolic_endocrin],20


In [28]:
val.head(1000)

,Vârstă,Ești,Unde locuiești?,Ocupatia,Care este înălțimea ta?,Care este greutatea ta actuala?,"Care este circumferința taliei tale, măsurata deasupra de ombilicului?",obezitate abdominala,rezistenta la insulina,prediabet,...,Care sunt obiectivele tale legate de sănătate? Ce îți dorești sa obții?,Greutatea la care vrei sa ajungi?,"In prezent, care este cea mai mare provocare a ta? Ce crezi ca te împiedica sa slăbești si sa ai o stare buna de sănătate?",De ce îți dorești sa slăbești?,Ai mai încercat in trecut sa slăbești?,Ce te-a împiedicat in trecut sa slăbești? De ce ai eșuat la alte încercări?,IMC,text_lemmatized,labels,scor_medical
1537,32,femeie,Iasi,Angajat,170.0,65.0,100.0,1,0,0,...,Sa dau jos din grasimea abdominala si sa ma po...,60,poftele necontrolate,"Nu ma simt bine in pielea mea, nu ma pot uita ...",1,"Nu am vazut rezultate, asa ca mi-a fost greu s...",22.5,menstruati neregulat constipatie durere cap po...,"[cardio_vascular, gastro_hepato_renal, neuro_p...",13
4152,35,femeie,Cluj-Napoca,Angajat,170.0,65.0,95.0,0,0,0,...,Sa arat ca un om normal. In principal sa scad ...,55.0,oboseala pofta de mancare si dulce,Pt ca ma simt letargica si nu vreau sa ma ingr...,Da,Oboseam. Mi se facea foame dupa 2-3 saptamani ...,22.5,ten gras acnee tiroidita hashimoto oboseala po...,"[metabolic_endocrin, neuro_psiho_energie]",16
3291,47,femeie,Jud Timiș,Angajat,158.0,73.0,110.0,1,0,0,...,Slăbit,60.0,nu mai am incredere ca ma poate ajuta cineva s...,Nu îmi place cum arăt nu mai pot să înbrac c...,Da,Nu erau rezultate și erau destul de costisitoa...,29.2,anxietate migren pt face injectie mai avea inc...,[neuro_psiho_energie],17
3901,38,femeie,Timișoara,Angajat,160.0,79.0,96.0,1,0,0,...,Sa slabesc,60.0,programul de mancat haotic,"Pentru a avea mai multa energie, pentru a nu f...",Da,"Am slăbit, dar am pus kilogramele înapoi. Acum...",30.9,program mancat haotic,[],7
2821,62,femeie,Iași,Pensionar,150.0,84.0,100.0,0,0,1,...,"Reglarea TA,greutate50kg,scădere țesutului adipos",50,lipsa de miscare intensa,"Nu vreau diabet,boli cardiovasculare",0,"Nu a fost cazul,analizele erau ok",37.3,colesterol ldl 190 lipsa miscare intensa,[],10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3239,22,femeie,Timișoara,Student,165.0,84.0,100.0,0,0,0,...,"Nu m-ar deranja să slăbesc câteva kg, dar prin...",50,toata viata am fost o persoana sportiva dar in...,"Scopul meu nu este neapărat de a slăbi, ci de ...",1,Când mi-am propus să slăbesc și am fost atentă...,30.9,cadere par dermatita durere menstural inaint i...,[],8
2399,53,femeie,Braila,Angajat,169.0,108.0,125.0,1,1,0,...,Un stil de viață sănătos care să mă ajute să s...,75,marea provocare ar fi sa reusesc sa mi schimb ...,"Să mă simt bine , să fiu activă",1,Probabil faptul ca datorită obiceiurilor mele ...,37.8,durere picior acid uric mare indic hom 8 77 ma...,[],20
1729,46,femeie,Galati,Antreprenor,172.0,72.0,86.0,1,0,0,...,Sa ma simt mai energica,62,programul haotic si neplanificatea riguroasa a...,Ptr fizic si sanatate,0,Nu am incercat,24.3,fibrom uterin program haotic neplanificate rig...,[],10
2477,45,femeie,Județul Iasi,Angajat,167.0,85.0,102.0,1,0,1,...,"Sa slăbesc,sa reduc valoarea la trigliceride, ...",68,sa slabesc nu stiu de ce nu reusesc,"Sa fiu bine cu sănătatea,sa nu mai obosesc la ...",1,Cred ca ....nu mai dau rezultate dietele,30.5,nici fel slabi sti reusi,[],15


In [29]:
test.head(1000)

,Vârstă,Ești,Unde locuiești?,Ocupatia,Care este înălțimea ta?,Care este greutatea ta actuala?,"Care este circumferința taliei tale, măsurata deasupra de ombilicului?",obezitate abdominala,rezistenta la insulina,prediabet,...,Care sunt obiectivele tale legate de sănătate? Ce îți dorești sa obții?,Greutatea la care vrei sa ajungi?,"In prezent, care este cea mai mare provocare a ta? Ce crezi ca te împiedica sa slăbești si sa ai o stare buna de sănătate?",De ce îți dorești sa slăbești?,Ai mai încercat in trecut sa slăbești?,Ce te-a împiedicat in trecut sa slăbești? De ce ai eșuat la alte încercări?,IMC,text_lemmatized,labels,scor_medical
2276,56,femeie,Deva,Antreprenor,158.0,83.0,111.0,1,1,1,...,Doresc sa reduc circumferita abdominala si sa ...,64 kg,ma lupt zi de zi cu greutatea sunt informata s...,Imi doresc sa slabesc pentru ca nu vreau sa aj...,1,Am slabit 30 de kg intr-un an! Apoi in sapte a...,33.2,vedere incetosata picior greu desi avea varic ...,"[cardio_vascular, metabolic_endocrin]",28
274,44,femeie,Bucuresti,Pensionar,165.0,84.0,102.0,1,0,1,...,Oprirea evoluției prediabet.,67kg,alte probleme de sanatate lips vointa si conse...,"Îmbunătățirea stării de sănătate., .",1,Am slăbit o singura data în 2001 în 2luni 1...,30.9,cardiaca mai prob problema sanatate lips voint...,[],22
3476,31,femeie,București,Angajat,159.0,70.0,90.0,0,1,0,...,"Vreau să am o stare de bine constantă, discipl...",55.0,lipsa consecventei in plus deseori imi propun ...,"Îmi doresc să mă simt bine în corpul meu, să a...",Da,"Diete care nu au avut efect, nu mi-au rezolvat...",27.7,menstruati neregulata dispozitie proasta insta...,[cardio_vascular],17
3922,43,femeie,Bucuresti,Antreprenor,150.0,80.0,96.0,1,0,0,...,"Sa nu mai am ficat gras, colesterol si triglic...",60.0,nu pot controla foamea poftele efectiv nu mai ...,Pentru sanatate si aspect fizic,Da,"Am reusit sa slabesc, dar la un moment dat am ...",35.6,avea simptom putea controla foame pofta efecti...,[],14
100,44,femeie,Anglia,Angajat,173.0,94.0,116.0,0,0,0,...,"Sa slabesc,sa am un stil de viață sănătos",63,timpul,Să fiu sănătoasă pentru fiica mea,1,Lipsa de timp și monotonia,31.4,durere spate articulatii timp,[],11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2971,45,femeie,Lussemburgo,Angajat,162.0,65.0,88.0,0,0,0,...,Să slăbesc 7 kg,58,lipsa de motivatie,Pt a redeveni cea care eram,1,Stresul,24.8,nici lipsa motivatie,[],7
3228,49,femeie,Ploiesti,Antreprenor,165.0,72.0,100.0,0,1,1,...,Rezistenta la insulina,63,o corectie a orelor si a nutrientilor din farf...,Sa mi reglez glicemia si sa ma simt bine in pi...,1,"Am slabit , in pandemie m am ingrasat si am to...",26.4,palpitatii corectie ora si nutrient farfurie,[],16
3056,45,femeie,cluj napoca,Angajat,167.0,81.0,96.0,0,1,0,...,sa slabesc,70,organizarea meselor timpul,sa ma simt mai bine,1,am slabit doar ca am pus la loc,29.0,organizare mesa timp,[],9
1498,27,femeie,"Com. Prundu, jud. Giurgiu",Altceva,168.0,84.0,90.0,1,0,0,...,"Sa slabesc, sa scad in centimetri",55,lipsa de energie,Pentru sanatate,1,Tentatii la orice pas,29.8,lipsa energie,[neuro_psiho_energie],11
